<a href="https://colab.research.google.com/github/Hemantr05/recommendation_systems/blob/main/collaborative_filtering_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!wget "https://files.grouplens.org/datasets/movielens/ml-25m.zip"
!unzip ml-25m.zip

--2022-01-10 13:10:29--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M   108MB/s    in 2.3s    

2022-01-10 13:10:32 (108 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
movies = pd.read_csv('ml-25m/movies.csv')
ratings = pd.read_csv('ml-25m/ratings.csv')

### Preprocessing

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field.

Let's remove the year from the title column by using pandas' replace function and store in a new year column.

Using regular expressions to find a year stored between parentheses
We specify the parantheses so we don't conflict with movies that have years in their titles

In [ ]:
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['year']

0        (1995)
1        (1995)
2        (1995)
3        (1995)
4        (1995)
          ...  
62418    (2018)
62419    (2001)
62420    (2018)
62421    (2001)
62422    (1962)
Name: year, Length: 62423, dtype: object

In [ ]:
#Removing paranthesis
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)
movies['year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
62418    2018
62419    2001
62420    2018
62421    2001
62422    1962
Name: year, Length: 62423, dtype: object

In [ ]:
#Removing the years from the 'title' column
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies['title']

0                          Toy Story 
1                            Jumanji 
2                   Grumpier Old Men 
3                  Waiting to Exhale 
4        Father of the Bride Part II 
                     ...             
62418                             We 
62419             Window of the Soul 
62420                      Bad Poems 
62421                   A Girl Thing 
62422        Women of Devil's Island 
Name: title, Length: 62423, dtype: object

In [ ]:
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies['title'] = movies['title'].apply(lambda x: x.strip())

In [ ]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
movies = movies.drop('genres', 1)
movies.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [ ]:
# Preprocessing ratings df
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
ratings = ratings.drop('timestamp', 1)
ratings.head()


,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


### Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .



In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [ ]:
#Filtering out the movies by title
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]

#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)

#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,164600,Akira,4.5
5,1968,"Breakfast Club, The",5.0


The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
141,2,1968,1.0
254,3,1,4.0
264,3,296,5.0


We now group up the rows by user ID.

In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

lets look at one of the users, e.g. the one with userID=1200

In [ ]:
userSubsetGroup.get_group(1200)

,userId,movieId,rating
167702,1200,1,5.0
167715,1200,1274,3.0


In [ ]:
#user with id 1200 having all 1 similar movies watched
len(userSubsetGroup.get_group(130))

1

In [ ]:
userSubsetGroup.get_group(110)

,userId,movieId,rating
14294,110,1,4.0


In [ ]:
#user with id 110 having only 1 similar movie watched
len(userSubsetGroup.get_group(110))

1

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [ ]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [ ]:
#Top most user with id 75 having all 5 similar moves watched
userSubsetGroup[0]

(43,       userId  movieId  rating
 5858      43        1     4.0
 5859      43        2     3.5
 5884      43      296     5.0
 5974      43     1274     4.5
 6018      43     1968     4.5)

In [ ]:
#name of top user group
userSubsetGroup[0][0]

43

In [ ]:
#dataframe of top user group
userSubsetGroup[0][1]

,userId,movieId,rating
5858,43,1,4.0
5859,43,2,3.5
5884,43,296,5.0
5974,43,1274,4.5
6018,43,1968,4.5


### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.
we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why [Pearson Correlation](https://www.questionpro.com/blog/pearson-correlation-coefficient/)?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![link text](https://www.questionpro.com/blog/wp-content/uploads/2020/04/pearson-formula.png)

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]


Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    #Get the N (total similar movies watched) for the formula 
    nRatings = len(group)
    
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    ###For Debugging Purpose
    #if nRatings<5:
    #    print(inputMovies['movieId'].isin(group['movieId'].tolist()))
    #    break
    #else:
    #    continue
    
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    
    #Now let's calculate the pearson correlation between two users, so called, x and y

    #For package based
    #scipy.stats import pearsonr
    #pearsonr(tempRatingList,tempGroupList)[0]

    #For hard code based
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/np.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

dict_items([(43, 0.9460287597114506), (171, 0.3288967572401365), (440, 0.9615384615384616), (597, 0.4688072309384957), (695, 0.8770580193070306), (757, 0.4385290096535153), (772, 0.584705346204686), (860, 0.8594395636904102), (939, 0.716114874039432), (1203, -0.41671450165524815), (1242, 0.711233325153824), (1401, 0.07520710469952328), (1652, 0.537086155529574), (1894, 0.6020183016345586), (1977, -0.323875137815648), (2345, 0.43852900965351493), (2429, 0.4385290096535153), (2469, 0.7161148740394331), (2640, -0.5264497328966636), (2766, -0.21926450482675575), (2982, 0.7307692307692307), (3150, -0.41602514716892186), (3266, 0.657793514480272), (3272, 0.0), (3274, 0.179028718509858), (3624, 0.6698641270570834), (3660, 0.7757911135427185), (3760, 0.5860090386731196), (4006, 0.179028718509858), (4019, 0.39666441401095853), (4060, 0.32328787506992), (4246, 0.89514359254929), (4410, 0.5288858853479448), (4420, 0.657793514480273), (4459, 0.8127665166512523), (4675, 0.537086155529574), (4975, 0

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.head()

,0
43,0.946029
171,0.328897
440,0.961538
597,0.468807
695,0.877058


In [ ]:
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.946029,43
1,0.328897,171
2,0.961538,440
3,0.468807,597
4,0.877058,695


### The top x similar users to input user

Now let's get the top 1000 users that are most similar to the input.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
99,0.983092,11495
64,0.970725,7723
55,0.962435,6550
94,0.961678,10960
63,0.961678,7571


### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [ ]:
topUsersRating = topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()


,similarityIndex,userId,movieId,rating
0,0.983092,11495,1,3.0
1,0.983092,11495,2,1.0
2,0.983092,11495,6,4.5
3,0.983092,11495,10,4.5
4,0.983092,11495,16,4.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.983092,11495,1,3.0,2.949275
1,0.983092,11495,2,1.0,0.983092
2,0.983092,11495,6,4.5,4.423913
3,0.983092,11495,10,4.5,4.423913
4,0.983092,11495,16,4.5,4.423913


In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,39.973168,152.887748
2,39.973168,98.535608
3,12.129644,31.955678
4,2.447945,5.662576
5,11.355650,23.548374


In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.824759,1
2,2.465044,2
3,2.634511,3
4,2.313196,4
5,2.073714,5


### Recommended movies
Now let's sort it and see the top 20 movies that the algorithm recommended!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
70927,5.0,70927
111235,5.0,111235
2931,5.0,2931
6660,5.0,6660
1169,5.0,1169


In [ ]:
movies.loc[movies['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())]

,movieId,title,year
295,299,Priest,1994
316,320,Suture,1993
1141,1169,American Dream,1990
1687,1759,"Four Days in September (O Que É Isso, Companhe...",1997
2839,2931,Time of the Gypsies (Dom za vesanje),1989
5008,5114,"Bad and the Beautiful, The",1952
6188,6300,Flickering Lights (Blinkende lygter),2000
6274,6394,Big Jake,1971
6287,6408,Animals are Beautiful People,1974
6460,6583,"Blood of Heroes, The (Salute of the Jugger, The)",1989


### Advantages and Disadvantages of Collaborative Filtering

**Advantages**

* Takes other user's ratings into consideration
* Doesn't need to study or extract information from the recommended item
* Adapts to the user's interests which might change over time

**Disadvantages**

* Approximation function can be slow
* There might be a low of amount of users to approximate
* Privacy issues when trying to learn the user's preferences